In [2]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)
import json

app = Flask(__name__)

secretFile = json.load(open("secretFile.txt",'r'))
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']

line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))


if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.19.0.5 - - [14/Aug/2020 13:35:37] "POST /callback HTTP/1.1" 200 -
172.19.0.5 - - [14/Aug/2020 13:35:38] "POST /callback HTTP/1.1" 200 -
172.19.0.5 - - [14/Aug/2020 13:35:44] "POST /callback HTTP/1.1" 200 -
172.19.0.5 - - [14/Aug/2020 13:35:45] "POST /callback HTTP/1.1" 200 -
172.19.0.5 - - [14/Aug/2020 13:36:06] "POST /callback HTTP/1.1" 200 -
172.19.0.5 - - [14/Aug/2020 13:36:06] "POST /callback HTTP/1.1" 200 -
172.19.0.5 - - [14/Aug/2020 13:36:22] "POST /callback HTTP/1.1" 200 -
172.19.0.5 - - [14/Aug/2020 13:36:23] "POST /callback HTTP/1.1" 200 -
